In [1]:

import os
import numpy as np
import pandas as pd
from copy import deepcopy as copy
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

import subprocess
import pickle
from scipy.stats import describe
from tqdm import tqdm

from astropy.io import fits

from lvmdrp.core import image
from lvmdrp.functions import imageMethod, rssMethod

import seaborn as sns

sns.set(context="talk", style="ticks", palette="husl", color_codes=True)

import warnings
warnings.filterwarnings("ignore")


def read_nist_reference(filename, wl_window):
    wli, wlf = wl_window
    ref_nist_or = pd.read_csv(
        filename, sep="\s+", comment="#").set_index("Wavelength")
    ref_nist_or = ref_nist_or.loc[(
        wli <= ref_nist_or.index) & (ref_nist_or.index <= wlf)]

    del_wl = np.diff(ref_nist_or.index).min()
    new_wl = np.unique(
        np.arange(wli, wlf+del_wl, del_wl).tolist()+ref_nist_or.index.to_list())
    ref_nist_rs = ref_nist_or.reindex(new_wl)

    return ref_nist_or, ref_nist_rs


In [2]:
from lvmdrp.main import load_master_config
from lvmdrp.utils.database import ReductionStatus, QualityFlag, create_or_connect_db, LVMFrames, get_raws_metadata


from lvmsurveysim.utils.sqlite2astropy import peewee2astropy


config = load_master_config()
create_or_connect_db(config)

lvm_frames = peewee2astropy(LVMFrames).to_pandas().set_index("id")

# 2246-2290 Darks (10 s, 30 s, 90 s)
# 2291-2335 Argon  (10 s, 30 s, 90 s)   
# 2336-2380 Xenon  (10 s, 30 s, 90 s)
# 2381-2425 Mercury-Argon  (10 s, 30 s, 90 s)
# 2426-2470 Neon  (10 s, 30 s, 90 s)
# 2471-2515 Mercury-Neon  (10 s, 30 s, 90 s)
targ_idxs = np.arange(2246, 2515+1, 1, dtype=int)
cont_idxs = np.asarray(list(range(2169,2183)) + list(range(2186,2200)))

In [3]:
cont_frames = lvm_frames.loc[lvm_frames.label.str.split("-").apply(lambda r: int(r[-1])).isin(cont_idxs)].sort_values("exptime", ascending=False)
targ_frames = lvm_frames.loc[lvm_frames.label.str.split("-").apply(lambda r: int(r[-1])).isin(targ_idxs)].sort_values("exptime", ascending=False)

cont_frames

,datetime,mjd,spec,ccd,exptime,imagetyp,obstime,observat,label,path,...,ffs,mi150,ts,ldls,nirled,reduction_started,reduction_finished,status,flags,calib_id
id,,,,,,,,,,,,,,,,,,,,,
12891,2022-11-23 07:48:04.021827,59646,sp1,b1,40.0,continuum,2022-03-08T03:36:24.532,LCO,sdR-s-b1-00002172,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,1,0,None,None,1,1,None
13400,2022-11-23 07:48:04.143532,59646,sp1,z1,40.0,continuum,2022-03-08T03:36:24.532,LCO,sdR-s-z1-00002172,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,1,0,None,None,1,1,None
13162,2022-11-23 07:48:04.129923,59646,sp1,b1,40.0,continuum,2022-03-08T03:47:11.477,LCO,sdR-s-b1-00002179,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,1,0,None,None,1,1,None
13628,2022-11-23 07:48:04.223037,59646,sp1,z1,40.0,continuum,2022-03-08T03:51:47.639,LCO,sdR-s-z1-00002182,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,1,0,None,None,1,1,None
13197,2022-11-23 07:48:04.132114,59646,sp1,r1,40.0,continuum,2022-03-08T03:31:47.178,LCO,sdR-s-r1-00002169,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,1,0,None,None,1,1,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13460,2022-11-23 07:48:04.147248,59646,sp1,r1,2.0,continuum,2022-03-08T04:05:08.622,LCO,sdR-s-r1-00002191,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,1,0,None,None,1,1,None
13463,2022-11-23 07:48:04.147411,59646,sp1,z1,2.0,continuum,2022-03-08T04:05:08.622,LCO,sdR-s-z1-00002191,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,1,0,None,None,1,1,None
13466,2022-11-23 07:48:04.147572,59646,sp1,r1,2.0,continuum,2022-03-08T04:02:26.310,LCO,sdR-s-r1-00002188,/home/mejia/Research/UNAM/lvm-drp/data.sdss5.o...,...,0,0,0,1,0,None,None,1,1,None


In [4]:
fiberflat_path = "_data-lvm/fiberflats"
if not os.path.isdir(fiberflat_path):
    os.makedirs(fiberflat_path, exist_ok=True)

ccds = ["b1", "r1", "z1"]
for idx, frame in tqdm(cont_frames.iterrows(), total=len(cont_frames), desc="processing continuum frames", ascii=True, unit="frame"):

    cont_path = frame.path
    cont_name = frame.label

    if os.path.isfile(f"{fiberflat_path}/{cont_name}.ms.fits"): continue

    imageMethod.preprocRawFrame_drp(
        in_image=cont_path,
        out_image=f"{fiberflat_path}/{cont_name}.pre.fits",
        boundary_x="1,2040",
        boundary_y="1,2040",
        positions="00,10,01,11",
        orientation="S,S,S,S"
    )

    imageMethod.LACosmic_drp(
        image=f"{fiberflat_path}/{cont_name}.pre.fits",
        out_image=f"{fiberflat_path}/{cont_name}.cosmic.fits",
        increase_radius=1, flim="1.3", parallel='5'
    )

    imageMethod.findPeaksAuto_drp(
        image=f"{fiberflat_path}/{cont_name}.pre.fits",
        out_peaks_file=f"{fiberflat_path}/{cont_name}.trace.peaks",
        disp_axis="X", threshold="5000", slice="3696", nfibers="41", median_box="1", median_cross="1", method="gauss", init_sigma="0.5", verbose=0
    )

    imageMethod.tracePeaks_drp(
        image=f"{fiberflat_path}/{cont_name}.pre.fits",
        peaks_file=f"{fiberflat_path}/{cont_name}.trace.peaks", trace_out=f"{fiberflat_path}/{cont_name}.trc.fits",
        steps=30, method="gauss", threshold_peak=50, poly_disp=5, coadd=30, verbose=0
    )

    imageMethod.subtractStraylight_drp(
        image=f"{fiberflat_path}/{cont_name}.pre.fits",
        trace=f"{fiberflat_path}/{cont_name}.trc.fits",
        stray_image=f"{fiberflat_path}/{cont_name}.back.fits",
        clean_image=f"{fiberflat_path}/{cont_name}.stray.fits",
        aperture=40, poly_cross=2, smooth_gauss=30
    )

    imageMethod.traceFWHM_drp(
        image=f"{fiberflat_path}/{cont_name}.stray.fits",
        trace=f"{fiberflat_path}/{cont_name}.trc.fits",
        fwhm_out=f"{fiberflat_path}/{cont_name}.fwhm.fits",
        blocks=32, steps=30, coadd=20, threshold_flux=50.0, poly_disp=5, clip="1.5,4.0"
    )

    imageMethod.extractSpec_drp(
        image=f"{fiberflat_path}/{cont_name}.stray.fits",
        trace=f"{fiberflat_path}/{cont_name}.trc.fits",
        out_rss=f"{fiberflat_path}/{cont_name}.ms.fits",
        fwhm=f"{fiberflat_path}/{cont_name}.fwhm.fits",
        parallel=5, method="optimal"
    )

processing continuum frames: 100%|##########| 84/84 [16:35:37<00:00, 711.16s/frame]   
